<a href="https://colab.research.google.com/github/dsaint31x/AOculus/blob/master/ML_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The CART Training Alogrithm

Scikit-Learn uses the *Classification And Regression Tree* (**CART**) alogrithms to train Decision Trees (also called "growing" tress). 

The algorithm works by first spliting the training set into two subsets using a single feature $k$ and a threshold $t_k$ (e.g., "petal length $\le$ 2.45cm").

How dose it choose $k$ and $t_k$?

It searches for the pair $(k,t_k)$ that **produces the pure subsets (weighted by their size).**

Equation 6-2 gives the cost function that the algorithm tries to minimize.

### Equation 6-2. CART cost function for classification

$$
J(k,t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}
$$

where
* $G_{left/right}$ measures the impurity of the left/right subset,
* $m_{left/right}$ is the number of instances in the left/right subset.

Once the CART algorithm has successfully split into the training set in two, it split the subsets using the same logic, then the sub-subsets, and so on, recursively. 

It stops recursing once it reaches the maximum depth (defined by the `max_depth` hyperparameter), or if it cannot find a split that will reduce impurity. 

A few other hyperparmeters (described in a moment) control additional stopping conditions (`min_samples_split`,`min_samples_leaf`, `min_weight_fraction_leaf`, and `max_leaf_nodes`)


> ### WARNING
>
> As you can see, the CART algorithm is a *greedy algorithm*: it greedily searches for an optimum split at the top level, then repeats the process at each subsequent level. It does not check whether or not the split will lead to the lowest possible impurity several levels down. A greedy algorithm often produces a solution that’s reasonably good but not guaranteed to be optimal.
> 
> Unfortunately, finding the optimal tree is known to be an *NP-Complete problem*: it requires $O(\exp(m))$ time, making the problem intractable even for small training sets. **This is why we must settle for a “reasonably good” solution.**


## Computational Complexity

Making predictions requires traversing the Decision Tree from the root to a leaf. 

Decision Trees generally are approximately balanced, so traversing the Decision Tree requires going through roughly $O(\log_2(m))$ nodes. 

Since each node only requires checking the value of one feature, the overall prediction complexity is $O(\log_2(m))$, **independent of the number of features**. So predictions are very fast, even when dealing with large training sets.


The training algorithm compares all features (or less if `max_features` is set) on all samples at each node. Comparing all features on all samples at each node results in a training complexity of $O(n \times m \log_2(m))$. 

For small training sets (less than a few thousand instances), Scikit-Learn can speed up training by presorting the data (set `presort=True`), but doing that slows down training considerably for larger training sets.

## Gini Impurity or Entropy?

By default, the Gini impurity measure is used, but you can select the **entropy** impurity measure instead by setting the criterion hyperparameter to "`entropy`". 

The concept of entropy originated in thermodynamics as a measure of molecular disorder: `entropy` **approaches zero when molecules are still and well ordered**. Entropy later spread to a wide variety of domains, including Shannon’s information theory, where it measures the average information content of a message: `entropy` is zero when all messages are identical. 
In Machine Learning, `entropy` is frequently used as **an impurity measure**: a set’s entropy is zero when it contains instances of only one class. 

Equation 6-3 shows the definition of the entropy of the i-th node. For example, the depth-2 left node in Figure 6-1 has an entropy equal to –$(49/54) \log_2 (49/54) – (5/54) \log_2 (5/54) \approx 0.445$.


### Equation 6-3. Entropy

$$
H_i = - \sum^n _{k=1\\p_{i,k} \ne 0} p_{i,k} \log_2 (p_{i,k})
$$

So, should you use Gini impurity or entropy? The truth is, most of the time it does not make a big difference: they lead to similar trees. 

* **Gini impurity is slightly faster to compute**, so it is a good default. However, when they differ, 
* **Gini impurity tends to isolate the most frequent class in its own branch of the tree**, while 
* **entropy tends to produce slightly more balanced trees**.

## Regularization Hyperparameters

Decision Trees make very few assumptions about the training data (as opposed to linear models, which assume that the data is linear, for example). If left **unconstrained**, the tree structure will adapt itself to the training data, fitting it very closely—indeed, most likely **overfitting** it. Such a model is often called a **nonparametric model**, not because it does not have any parameters (it often has a lot) but **because the number of parameters is not determined prior to training**, so the model structure is free to stick closely to the data. 

In contrast, *a **parametric model**, such as a linear model*, has a **predetermined number of parameters**, so **its degree of freedom is limited**, reducing the risk of overfitting (but increasing the risk of underfitting).

**To avoid overfitting** the training data, you need to **restrict the Decision Tree’s freedom during training**. As you know by now, this is called regularization. The regularization hyperparameters depend on the algorithm used, but generally you can at least restrict the maximum depth of the Decision Tree. In Scikit-Learn, this is controlled by the max_depth hyperparameter (the default value is None, which means unlimited). Reducing max_depth will regularize the model and thus reduce the risk of overfitting.

The `DecisionTreeClassifier` class has a few other parameters that similarly restrict the shape of the Decision Tree: 
* `min_samples_split` (the minimum number of samples a node must have before it can be split), 
* `min_samples_leaf` (the minimum number of samples a leaf node must have), 
* `min_weight_fraction_leaf` (same as min_samples_leaf but expressed as a fraction of the total number of weighted instances), 
* `max_leaf_nodes` (the maximum number of leaf nodes), and 
* `max_features` (the maximum number of features that are evaluated for splitting at each node). 

Increasing min_* hyperparameters or reducing max_* hyperparameters will regularize the model.

>
> ### NOTE
>
> Other algorithms work by first training the **Decision Tree** without restrictions, then **pruning** (deleting) unnecessary nodes. 
> A node whose children are all leaf nodes is considered unnecessary if **the purity improvement it provides is *not statistically significant*.** 
>
> Standard statistical tests, such as 
> * the χ2 test (chi-squared test), 
>   * are used to estimate the probability that the improvement is purely the result of chance (which is called the null hypothesis). 
>   * If this probability, called the **p-value**, is higher than a given threshold (typically 5%, controlled by a hyperparameter), then the node is considered unnecessary and its children are deleted. 
> 
> The pruning continues until all unnecessary nodes have been pruned.

Figure 6-3 shows two Decision Trees trained on the moons dataset (introduced in Chapter 5). On the left the Decision Tree is trained with the default hyperparameters (i.e., no restrictions), and on the right it’s trained with `min_samples_leaf=4`. 

It is quite obvious that the model on the left is overfitting, and the model on the right will probably generalize better.

![mls2_0603.png](./fig/mls2_0603.png)




